In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from kmodes.kprototypes import KPrototypes

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load the Dataset

In [ ]:
vg_df = pd.read_csv('../input/video-game-sales-with-ratings/Video_Games_Sales_as_at_22_Dec_2016.csv')

In [ ]:
# Looking in to the data
vg_df.head(5)

In [ ]:
# Looking at the dataframe info
vg_df.info()

There seems to be some missing data. Let's explore that

# Missing Data

In [ ]:
# Calculate the missing values percentage in each column
missing_total = vg_df.isnull().sum().sort_values(ascending=False)
percent = ((missing_total*100)/(vg_df.isnull().count())).sort_values(ascending=False)
percent

As our primary goal is visualisation, we will drop the rows wherever there is some missing data.

# Dropping the rows with NA

In [ ]:
vg_df.dropna(axis=0, inplace=True)
vg_df.info()

Now, all the columns are having data and no missing values are present.

In [ ]:
vg_df.head()

In [ ]:
# Looking the datatypes of the columns
vg_df.info()

As the year of relase is a date column and not a continuous column, we will convert it in to a object and pass to the clustering algorithm.

The User_Score column should also be converted into float from objectg.

In [ ]:
# Converting the datatype of year_of_relase and User_score columns
vg_df['Year_of_Release'] = vg_df['Year_of_Release'].astype(str)
vg_df['User_Score'] = vg_df['User_Score'].astype(float)
vg_df.info()

In [ ]:
vg_df.head()

In [ ]:
# Looking at the no.of unique values in the categorical columns
vg_df.select_dtypes('object').nunique()

As we have high number of unique values in the Name, Developer and Publisher columns we will drop as they are no good for clustering.

In [ ]:
# Dropping the unnecesasry columns for clustering.
vg_df1 = vg_df.drop(['Name','Developer', 'Publisher'], axis=1)
vg_df1.head()

Before we scale the data we must reset the index, so that we can this index to join back the scaled numerical columned dataframe with the original datafram with categorical columns

In [ ]:
vg_df1.reset_index(inplace=True, drop=True)
vg_df1.head()

### Selecting the categorical columns as a separate dataframe so that we can use this to join with scaled numerical columns later

In [ ]:
vg_df_cat = vg_df1[['Platform', 'Year_of_Release', 'Genre', 'Rating']]
vg_df_cat.head()

Let's select the numerical columns and scale them

# Scaling the Data

As we have the numerical columns ranging the data in differnt ranges, we need to scale them. For this purpose we will use the MinMaxScaler()

In [ ]:
# Selecting the numerical columns for scaling
vg_df_num = vg_df1[['NA_Sales', 'EU_Sales', 'JP_Sales','Other_Sales','Global_Sales','Critic_Score','Critic_Count','User_Score','User_Count']]
vg_df_num.head()

### Extracting the column names of the dataframe vg_df_num

Next we have to scale the data in vg_df_num. But while doing that we will get the vg_df_num as numpy array. So, in order to convert again it to a dataframe we will first extract the column names of the dataframe vg_df_num before scaling.

In [ ]:
vg_df_num_columns = list(vg_df_num.columns)
print(vg_df_num_columns)

In [ ]:
# mix-max scale the data between 0 and 1
mms = MinMaxScaler()
vg_df_num = mms.fit_transform(vg_df_num)

In [ ]:
vg_df_num

### Converting vg_df_num back to dataframe

As seen in the above output vg_df_num is now converted in to a numpy array. So, we will convert it back to the Dataframe back, so that we can visualize the data in some columns before and after scaling.

In [ ]:
# converting the X to dataframe
vg_df_num = pd.DataFrame(vg_df_num)
vg_df_num.columns = vg_df_num_columns
vg_df_num.head()

Now lets join the data dataframes vg_df_num and vg_df_cat

In [ ]:
# Joining the scaled dataframe and categorical dataframe
vg_df_scaled = pd.concat([vg_df_cat, vg_df_num], axis=1)
vg_df_scaled.head()

# Clustering

Let's try to cluster the data and find out the similar featured observations that can fall in to similar group.

As we have categorical and numerical variables for clustering, we will use the KPrototypes clustering.

As the kprototypes clustering algorithm needs the explicit positions of the categorical columns, lets try to find them.

In [ ]:
# Get the position of categorical columns
catColumnsPos = [vg_df_scaled.columns.get_loc(col) for col in list(vg_df_scaled.select_dtypes('object').columns)]
print('Categorical columns           : {}'.format(list(vg_df_scaled.select_dtypes('object').columns)))
print('Categorical columns position  : {}'.format(catColumnsPos))

In [ ]:
# Converting the dataframe in to a numpy array
vg_df_array = vg_df_scaled.values
vg_df_array

### explicitly mentioning the numerical columns by converting them to float datatype

In [ ]:
vg_df_array[:, 4:13] = vg_df_array[:, 4:13].astype(float)

In [ ]:
vg_df_array

# kprototypes clustering implementation

# Finding the Optimal number of clusters

In [ ]:
# Trying the range of clusters from 1 to 10
cost = []
for cluster in range(1, 10):
    kprototype = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Huang', random_state = 0)
    kprototype.fit_predict(vg_df_array, categorical = catColumnsPos)
    cost.append(kprototype.cost_)
    print('Cluster initiation: {}'.format(cluster))

In [ ]:
# Converting the results into a dataframe and plotting them
df_cost = pd.DataFrame({'Cluster':range(1, 10), 'Cost':cost})
df_cost

In [ ]:
# Plotting the clusters to cost function.
sns.lineplot(x='Cluster', y='Cost', data=df_cost)

In the above plot, by using the elbow method we can clearly see an elbow at 3 clusters. The change for every unit increase in the clusters number in the cost is not substantial. So, the optimal number of clusters is going to be 3. 

In [ ]:
# Implementing kprototype algorithm
kproto = KPrototypes(n_clusters=3, verbose=2, max_iter=20)
clusters = kproto.fit_predict(vg_df_array, categorical=[0,1,2,3])

In [ ]:
# Look at the cluster centroids
print(kproto.cluster_centroids_)

In [ ]:
# printing all the clustered numbers
cluster_dict=[]
for c in clusters:
    cluster_dict.append(c)
print(cluster_dict)

In [ ]:
# Attach the clustering output to the original dataframe vg_df_scaled
vg_df_scaled['cluster']=cluster_dict
vg = vg_df_scaled
vg.head()

In [ ]:
# Attach the clustering output to the original dataframe vg_df
vg_df['cluster']=cluster_dict
vg = vg_df
vg.head()

In [ ]:
# Looking at the cluster-0
vg[vg['cluster']==0].head()

In [ ]:
# Looking at the cluster-1
vg[vg['cluster']==1].head()

In [ ]:
# Looking at the cluster-0
vg[vg['cluster']==2].head()

# Exporting the dataframe as a CSV

Let's export the cleaned dataframe to a csv file

In [ ]:
vg.to_csv('videogames_sales_cleaned.CSV')